In [1]:
from pathlib import Path
import h5py

In [2]:
def recursive_load_from_h5py(file, group_path="/"):
    group = file[group_path]
    if isinstance(group, h5py.Dataset):
        return group[()]  # Read dataset value
    elif isinstance(group, h5py.Group):
        if all(k.isdigit() for k in group.keys()):  # Likely a list
            return [
                recursive_load_from_h5py(file, f"{group_path}/{k}")
                for k in sorted(group.keys(), key=int)
            ]
        else:  # Dictionary-like group
            return {
                k: recursive_load_from_h5py(file, f"{group_path}/{k}")
                for k in group.keys()
            }
    else:
        raise TypeError(f"Unsupported group type: {type(group)}")

def load_from_h5py(file_path: str | Path, group_path="/"):
    """
    Load a pytree structure from an HDF5 file.

    Args:
        file (h5py.File): An open HDF5 file object.
        group_path (str): The HDF5 group path to read data from.

    Returns:
        The reconstructed data structure.
    """
    with h5py.File(file_path, "r") as file:
        return recursive_load_from_h5py(file, group_path)
    
def save_to_h5(mocap_data, file_path):
    with h5py.File(file_path, 'w') as hf:
        hf.create_dataset('tracks', data=mocap_data)

In [3]:
mocap_data = load_from_h5py("scaled_down_stick_bug.h5")

In [4]:
kp_data = mocap_data['tracks']

In [5]:
kp_data.shape

(16875, 3, 50)

In [6]:
reshaped_kp_data = kp_data[:8000, :, :]

In [7]:
reshaped_kp_data.shape

(8000, 3, 50)

In [8]:
save_to_h5(reshaped_kp_data, "8000_f_stick_bug.h5")